# Setting up a latent function Gaussian Process

Let us start by importing Turing.jl, the library we will use to write our statistical model and our GaussianProcess.jl package. \
The rest of the imports will only be used to generate the data and plot. 

In [1]:
using Turing
using Distances
using LinearAlgebra
using LimberJack

    CondaPkg Found dependencies: /Users/vis/.julia/packages/CondaPkg/0UqYV/CondaPkg.toml
    CondaPkg Found dependencies: /Users/vis/.julia/packages/PythonCall/mkWc2/CondaPkg.toml
    CondaPkg Found dependencies: /Users/vis/.julia/packages/IJulia/y8YGF/CondaPkg.toml
    CondaPkg Initialising pixi
             │ /Users/vis/.julia/artifacts/d2fecc2a9fa3eac2108d3e4d9d155e6ff5dfd0b2/bin/pixi
             │ init
             │ --format pixi
             └ /Users/vis/Desktop/Cosmography_GP/.CondaPkg
✔ Created /Users/vis/Desktop/Cosmography_GP/.CondaPkg/pixi.toml
    CondaPkg Wrote /Users/vis/Desktop/Cosmography_GP/.CondaPkg/pixi.toml
             │ [dependencies]
             │ openssl = ">=3, <3.6"
             │ jupyter_kernel_test = "*"
             │ jupyter_client = "*"
             │ 
             │     [dependencies.python]
             │     channel = "conda-forge"
             │     build = "*cp*"
             │     version = ">=3.9,<4"
             │ 
             │ [project]
     

In [2]:
using Interpolations
using GaussianProcess
using Plots
using QuadGK

### 1. Get the Hz data
This time we will apply a non-linear transformation to our data.

In [3]:
function CC()
    z = [
        0.07, 0.09, 0.12, 0.17, 0.179, 0.199, 0.2,
        0.27, 0.28, 0.352, 0.38, 0.3802, 0.4,
        0.4004, 0.4247, 0.44, 0.4497, 0.47, 0.4783,
        0.48, 0.51, 0.593, 0.6, 0.61, 0.68, 0.73,
        0.781, 0.875, 0.88, 0.9, 1.037, 1.3,
        1.363, 1.43, 1.53, 1.75, 1.965
    ]

    data = [
        69.0, 69.0, 68.6, 83.0, 75.0, 75.0, 72.9,
        77.0, 88.8, 83.0, 81.5, 83.0, 95.0, 77.0,
        87.1, 82.6, 92.8, 89.0, 80.9, 97.0, 90.4,
        104.0, 87.9, 97.3, 92.0, 97.3, 105.0,
        125.0, 90.0, 117.0, 154.0, 168.0, 160.0,
        177.0, 140.0, 202.0, 186.5
    ]

    err = [
        19.6, 12.0, 26.2, 8.0, 4.0, 5.0, 29.6,
        14.0, 36.6, 14.0, 1.9, 13.5, 17.0, 10.2,
        11.2, 7.8, 12.9, 23.0, 9.0, 62.0, 1.9,
        13.0, 6.1, 2.1, 8.0, 7.0, 12.0, 17.0,
        40.0, 23.0, 20.0, 17.0, 33.6, 18.0,
        14.0, 40.0, 50.4
    ]

    cov = zeros(length(z), length(z))
    for i in 1:length(z)
        cov[i, i] = err[i]^2
    end

    return (
        data_name = "CC",
        z = z,
        data = data,
        cov = cov
    )
end


CC (generic function with 1 method)

In [4]:
function BOSSDR12()
    z = [0.38, 0.51, 0.61]
    data = [81.2087, 90.9029, 98.9647]
    cov = [5.00049e+02 2.94536e+02 1.42011e+02; 2.94536e+02 7.02299e+02 4.32750e+02; 1.42011e+02 4.32750e+02 1.01718e+03]
    return (data_name = "BOSSDR12", z = z, data = data, cov = cov)
end

BOSSDR12 (generic function with 1 method)

In [5]:
function BOSSDR12_dm()
    z = [0.38, 0.51, 0.61]
    data = [1512.39, 1975.22, 2306.68]
    cov = [3.63049e+00 1.80306e+00 9.19842e-01; 1.80306e+00 3.77146e+00 2.21471e+00; 9.19842e-01 2.21471e+00 4.37982e+00]
    return (data_name = "BOSSDR12_dm", z = z, data = data, cov = cov)
end

BOSSDR12_dm (generic function with 1 method)

In [6]:
cc = CC();
bossdr12 = BOSSDR12();
bossdr12dm = BOSSDR12_dm();

In [7]:
hubble_data  = [bossdr12.data; cc.data; bossdr12dm.data];
zh = [bossdr12.z; cc.z; bossdr12dm.z];

In [8]:
covariance = zeros(length(bossdr12.data)+ length(cc.data)+ length(bossdr12dm.data), 
length(bossdr12.data)+ length(cc.data)+ length(bossdr12dm.data));
for i in 1:length(bossdr12.data)
    for j in 1:length(bossdr12.data)
        covariance[i,j] = bossdr12.cov[i,j]
    end
end

for i in 1:length(cc.data)
    for j in 1:length(cc.data)
        covariance[i+length(bossdr12.data), j+length(bossdr12.data)] = cc.cov[i,j]
    end
end

for i in 1:length(bossdr12dm.data)
    for j in 1:length(bossdr12dm.data)
        covariance[i+length(bossdr12.data)+length(bossdr12dm.data), j+length(bossdr12.data)+length(bossdr12dm.data)] = cc.cov[i,j]
    end
end


In [9]:
zcont = range(0, 2.5, length=24);
cosmo1 = Cosmology();
H1 = cosmo1.cpar.h*100*Ez(cosmo1, zcont);

In [10]:
omegam = 0.3
omegab = 0.05
hc = 0.67

0.67

### 2. Check what our priors look like

In [11]:
function model_latent_GP(eta, l, v;  
                         x=zcont, z=zh, data_cov=covariance)
    # Dimensions of predictors .
    kernel = sqexp_cov_fn(x; eta=eta, l=l)
    cpar = CosmoPar(Ωm=omegam,  Ωb=omegab, h=hc)
    mean_hx = cpar.h*100*Ez(cpar, x)
    hx = latent_GP(mean_hx, v, kernel)
    hz =  conditional(x, z, hx, sqexp_cov_fn;
                     eta=eta, l=l)
    return hx, hz
end

model_latent_GP (generic function with 1 method)

In [12]:
xx = LinRange(0, 1, 300);

In [13]:
N_samples = 100
hxs=zeros(N_samples, 24)
hzs=zeros(N_samples, 300)
for i in 1:N_samples
    eta = 200*rand(Uniform(0., 0.1))
    l = 0.02*rand(Uniform(0.1, 3))
    v = rand(MvNormal(zeros(length(zcont)), ones(length(zcont))))
    hxs[i, :], hzs[i, :] = model_latent_GP(eta, l, v; z=xx)
end

y_m, y_s = mean(hzs, dims=1), std(hzs, dims=1);
gp_m, gp_s = mean(hxs, dims=1), std(hxs, dims=1);

hz_itp = linear_interpolation(zs, Vector(y_s), extrapolation_bc=Line())

MethodError: MethodError: no method matching (Vector)(::Matrix{Float64})
The type `Vector` exists, but no method is defined for this combination of argument types when trying to construct it.

You might have used a 2d row vector where a 1d column vector was required.
Note the difference between 1d column vector [1,2,3] and 2d row vector [1 2 3].
You can convert to a column vector with the vec() function.

Closest candidates are:
  (Array{T, N} where T)(::AbstractArray{S, N}) where {S, N}
   @ Core boot.jl:603
  (Vector)(!Matched::UndefInitializer, !Matched::Integer)
   @ Base baseext.jl:32
  (Vector)(!Matched::SparseArrays.CHOLMOD.Dense{T}) where T
   @ SparseArrays /opt/homebrew/Cellar/julia/1.11.7_1/share/julia/stdlib/v1.11/SparseArrays/src/solvers/cholmod.jl:1144
  ...


In [ ]:
function comoving_d(zs, hz_itp nz)
    chis = zeros(Real, nz)
    for i in 1:nz
        zz = zs[i]
        chis[i] = quadgk(z -> 1.0/hz_itp(z), 0.0, zz, rtol=1E-5)[1]
        #chis[i] = quadgk(z -> 1.0/Ez(cpar, z), 0.0, zz, rtol=1E-5)[1]
        #chis[i] *= CLIGHT_HMPC / cpar.h

    end
    chii = linear_interpolation(zs, Vector(chis), extrapolation_bc=Line())
    return chii
end


comoving_d (generic function with 1 method)

In [ ]:
# plot(zh, hubble_data, label="Data", ms=2, seriestype=:scatter)
# plot!(zcont, H1)
# plot!(zcont, vec(gp_m), label="GP mean")
# plot!(zcont, vec(gp_m .- gp_s),  fillrange = vec(gp_m .+ gp_s), fillalpha=0.2, c=1, label="GP standard deviation")

In [ ]:
# plot(zh, hubble_data, label="Data", ms=2, seriestype=:scatter)
# plot!(zcont, H1)
# plot!(zh, vec(y_m), label="y mean")
# plot!(zh, vec(y_m .- y_s),  fillrange = vec(y_m .+ y_s), fillalpha=0.2, c=1, label="y standard deviation")

### 3. Define our statistical model

In [15]:
@model function stats_model(y; X=zcont, data_x=zh, data_cov=covariance)
    # Priors.
    eta ~ 200*Uniform(0.0, 0.1)
    l ~ 0.02*Uniform(0.1, 3)
    v ~ MvNormal(zeros(length(X)), ones(length(X)))
    kernel = sqexp_cov_fn(X, eta=eta, l=l)
    cpar = CosmoPar(Ωm=omegam,  Ωb=omegab, h=hc)
    mean_hz = cpar.h*100*Ez(cpar, X)
    gp = latent_GP(mean_hz, v, kernel)
    m = conditional(X, data_x, gp, sqexp_cov_fn; eta=eta, l=l)
    y ~ MvNormal(m, data_cov)
end

stats_model (generic function with 2 methods)

### 4. Sample the model

In [17]:
chain = sample(stats_model(hubble_data), NUTS(100, 0.65), 1000)

Sampling   0%|                                          |  ETA: N/A
Sampling 100%|██████████████████████████████████████████| Time: 0:00:00


PosDefException: PosDefException: matrix is not positive definite; Factorization failed.

### 5. Check how our posteriors look like

In [ ]:
eta_p = group(chain, :eta).value.data[:, :, 1];
l_p = group(chain, :l).value.data[:, :, 1];
v_p = group(chain, :v).value.data[:, :, 1];

In [ ]:
println(length(v_p))

In [ ]:
N_samples2 = length(eta_p)
gps2=zeros(N_samples2, 24)
ys2=zeros(N_samples2, 100)
for i in 1:N_samples2
    gps2[i, :], ys2[i, :] = model_latent_GP(eta_p[i], l_p[i], v_p[i, :];
                                                data_x=LinRange(0, 2.3, 100))
end

y_m2, y_s2 = mean(ys2, dims=1), std(ys2, dims=1);
gp_m2, gp_s2 = mean(gps2, dims=1), std(gps2, dims=1);

In [ ]:
println(y_m2)

In [ ]:
plot(zh, hubble_data, label="Data", ms=2, seriestype=:scatter)
plot!(zcont, H1, label="LCDM", lc=:black)
plot!(LinRange(0, 2.3, 100), vec(y_m2), label="Y mean")
plot!(LinRange(0, 2.3, 100), vec(y_m2 .- y_s2),  fillrange = vec(y_m2 .+ y_s2), fillalpha=0.2, c=1, label="Y standard deviation")


In [ ]:
plot(zh, hubble_data, label="Data", ms=2, seriestype=:scatter)
plot!(zcont, H1, label="LCDM", lc=:black)
plot!(zcont, vec(gp_m2), label="GP mean")
plot!(zcont, vec(gp_m2 .- gp_s2),  fillrange = vec(gp_m2 .+ gp_s2), fillalpha=0.2, c=1, label="GP standard deviation")